In [ ]:
#import necessary libraries
import sempy.fabric as fabric
import pandas as pd
import numpy as np
import re
from notebookutils import mssparkutils

# set temporary config for date issue
spark.conf.set("spark.sql.parquet.datetimeRebaseModeInWrite", "CORRECTED")
spark.conf.set("spark.sql.parquet.datetimeRebaseModeInRead", "CORRECTED")

##########################################################################################################
##                    THIS IS THE SECTION THAT REQUIRES INITIAL CONFIGURATION BY THE USER
##########################################################################################################
########################################################################################
###   MANUAL CONFIG HERE
#######################################################################################
#Define lakehouse and schema names
workspace_name = "amaser_dp700" #this is the workspace name where the notebook runs, and where the lakehouse lives
lakehouse_name = "testthislakehouseoption"
lakehouse_description = "Model Documentation Lakehouse"  # this is only used if the notebook creates a new lakehouse
schema_name = "docuschema"  # leave this blank if using the built in dbo schema, or NOT using the schema enabled lakehouse


# Define colors for table navigator page
factcolor = '#A6B916'
dimcolor = '#C8B78A'
defaultcolor = '#A66999'

#Create list of workspaces and Models.  This allows users to limit which models get documented.  A user could also
#   tweak this notebook, to use sempy functions, and query all workspaces/models that he/she has access to, if that's preferred.
#THE ORDER OF THE WORKSPACES AND MODELS MUST BE THE SAME
#  ie the third report MUST be in the third worksapce
data = {
    'Workspace': ['Industry Demos', 'Industry Demos','PTurley AzureDevOps Integration'],
    'Model': ['Insurance Power BI Demo', 'Product Model','Airline Performance']
}
###########################################################################################
workspace_id = fabric.get_workspace_id()

#######################################################
## Check for existing items
######################################################
## check if lakehouse exists
try:
    lakehouse_object = mssparkutils.lakehouse.get(lakehouse_name)
    lakehouse_id=lakehouse_object.id
except Exception as e:
    try:

        fabric.create_lakehouse(display_name=lakehouse_name,description=lakehouse_description,workspace=workspace_name)
        lakehouse_object = mssparkutils.lakehouse.get(lakehouse_name)
        lakehouse_id=lakehouse_object.id
    except Exception as e:
        print('Unable to create lakehouse')
        print(e)



##create schema if not exists
if schema_name == "":
    lakehouse_value = f'{lakehouse_name}.'
else:
    #if fabric.lakehouse.schema_exists(schema_name,lakehouse=lakehouse_id,workspace=workspace_id):
    #    lakehouse_value = f'{workspace_name}.{lakehouse_name}.{schema_name}.'
    #else:
       
            #schema_path = f"abfss://{workspace_id}@onelake.dfs.fabric.microsoft.com/{lakehouse_id}/Tables/{schema_name}"
            #mssparkutils.fs.mkdirs(schema_path)
            schemaquery = f"CREATE SCHEMA IF NOT EXISTS {workspace_name}.{lakehouse_name}.{schema_name} "
            spark.sql(schemaquery)
            lakehouse_value = f'{workspace_name}.{lakehouse_name}.{schema_name}.'
        

print(lakehouse_value)

#create the log table. 
logquery = f'CREATE TABLE IF NOT EXISTS {lakehouse_value}modelLog (timeval TIMESTAMP, table_name STRING, Message STRING) USING DELTA'
#print(logquery)
spark.sql(logquery)

#function to log activity.  This function can be tweaked to add lakehouse/schema name, so all the code in the lower cells don't need to be adjusted
def logActivity(table_name,message):
    logquery = f'INSERT INTO {lakehouse_value}modellog (timeval,table_name,Message) VALUES (CURRENT_TIMESTAMP(),"{table_name}","{message}")'
    #print(f'log query={logquery}')
    #print(logquery)
    spark.sql(logquery)

#function to write the dataframe as a database
def writeTable(pd_df, tablename):
  if len(pd_df) != 0:
    query = f"drop table if exists {lakehouse_value}{tablename}"
    spark.sql(query)
    spark_df = spark.createDataFrame(pd_df)
    full_table_name = f'{lakehouse_value}{tablename}'
    #print(full_table_name)
    try:
        spark_df.write.format("delta").mode("overwrite").option("overwriteSchema","true").saveAsTable(full_table_name)
        logActivity(tablename,f'{tablename} written to lakehouse')
    except Exception as e:
        logActivity(tablename,f'Failed to write {tablename}')
        

####note
# You must attach the requested lakehouse to this notebook, for it to work properly.
#     If you are using schema enabled lakehousse, you'll want to modify the writeTable function to write to f'{lakehouse}.{schema}.{tablename}'abs
#      to account for those options. You'll need to update the query variable as well

#######################################################################################################
## from this section onward, there shouldn't be any updating necessary
#######################################################################################################
# add in the path column.  this isn't strictly necessary
df = pd.DataFrame(data)
df['WorkspacePath'] = f'powerbi://api.powerbi.com/v1.0/myorg/'+ df['Workspace']
df['semanticmodel_id'] = df.index + 1 # addsan auto increment index to  the table.
logActivity('SemanticModel','SemanticModel processing started')


####################################################################################################################


#DEFINE FUNCTIONS

#function to get data from model
def getMetaData(daxquery,fieldname,indexfield):
    indexname = indexfield+'_id'
    #build out first row
    model_df = fabric.evaluate_dax(
        workspace=df.iloc[0,0],
        dataset=df.iloc[0,1],
        dax_string=daxquery
    )
    model_df['semanticmodel_id'] = df.iloc[0,3]
    model_df['SemanticModel'] = df.iloc[0,1]
    #cycle through remaining rows, if more than 1 row
    if len(df) > 1:
        for index,row in df.iloc[1:].iterrows():
            temp_df = fabric.evaluate_dax(
                workspace=df.iloc[index,0],
                dataset=df.iloc[index,1],
                dax_string=dax_query
            )
            temp_df['semanticmodel_id']=df.iloc[index,3]
            temp_df['SemanticModel'] = df.iloc[index,1]
            #print('temp df executed')
            #append most recent
            model_df = pd.concat([model_df,temp_df],ignore_index=True)

    model_df.columns = model_df.columns.str.replace('[', '')
    model_df.columns = model_df.columns.str.replace(']', '')
    model_df = model_df.rename(columns={'Name':fieldname})
    model_df[indexname] = model_df.index + 1
    return model_df


# Function to extract hashtags from a string
def extract_hashtags(description):
    if isinstance(description, str):  # Check if description is a string
        hashtags = re.findall(r'%%(\w+)', description)
        return hashtags
    else:
        return []




StatementMeta(, 7f293312-2244-4a98-b187-e4258ae663d2, 10, Finished, Available, Finished)

amaser_dp700.testthislakehouseoption.docuschema.


log query=INSERT INTO amaser_dp700.testthislakehouseoption.docuschema.modellog (timeval,table_name,Message) VALUES (CURRENT_TIMESTAMP(),"SemanticModel","SemanticModel processing started")


Process list of semantic models from the user-defined list. save as a table in the lakehouse

In [ ]:
#Write Semantic Model table
writeTable(df,"SemanticModel")
logActivity('SemanticModel','SemanticModel processing completed')

StatementMeta(, 7f293312-2244-4a98-b187-e4258ae663d2, 11, Finished, Available, Finished)

amaser_dp700.testthislakehouseoption.docuschema.SemanticModel


log query=INSERT INTO amaser_dp700.testthislakehouseoption.docuschema.modellog (timeval,table_name,Message) VALUES (CURRENT_TIMESTAMP(),"SemanticModel","SemanticModel written to lakehouse")


log query=INSERT INTO amaser_dp700.testthislakehouseoption.docuschema.modellog (timeval,table_name,Message) VALUES (CURRENT_TIMESTAMP(),"SemanticModel","SemanticModel processing completed")


Process Model query.  Save Model data to the lakehouse

In [12]:
#Get Model Data
logActivity('Model','Model processing started')
dax_query = "EVALUATE INFO.MODEL()"
model_df = getMetaData(dax_query,'ModelName','model')
#display(model_df)

writeTable(model_df,"Model")
logActivity('Model','Model processing completed')

StatementMeta(, 7f293312-2244-4a98-b187-e4258ae663d2, 12, Finished, Available, Finished)

log query=INSERT INTO amaser_dp700.testthislakehouseoption.docuschema.modellog (timeval,table_name,Message) VALUES (CURRENT_TIMESTAMP(),"Model","Model processing started")


amaser_dp700.testthislakehouseoption.docuschema.Model


log query=INSERT INTO amaser_dp700.testthislakehouseoption.docuschema.modellog (timeval,table_name,Message) VALUES (CURRENT_TIMESTAMP(),"Model","Model written to lakehouse")


log query=INSERT INTO amaser_dp700.testthislakehouseoption.docuschema.modellog (timeval,table_name,Message) VALUES (CURRENT_TIMESTAMP(),"Model","Model processing completed")


Process Tables dax query to collect table data from all defined models

In [ ]:
#get Table Data
logActivity('PBITables','Table processing started')
dax_query = """EVALUATE
VAR __tables =
    INFO.TABLES ()
VAR __storagemode =
    SELECTCOLUMNS ( INFO.VIEW.TABLES (), \"ID\", [ID], \"StorageMode\", [StorageMode] )
VAR __combined =
    NATURALLEFTOUTERJOIN ( __tables, __storagemode )
RETURN
    __combined

"""
table_df = getMetaData(dax_query,'TableName','table')

#add table type column for slicer. Cleanup column names and create combined key for joining
table_df['TableType']=table_df['Description'].apply(lambda x:extract_hashtags(x))
table_df['TableType']=table_df['TableType'].astype(str)
table_df['TableType'] = table_df['TableType'].str.removeprefix('[\'')
table_df['TableType'] = table_df['TableType'].str.replace('\']', '')
table_df['TableType'] = table_df['TableType'].str.replace(']', '')
table_df['TableType'] = table_df['TableType'].str.replace('[', '')
table_df['compoundkey'] = table_df['SemanticModel'] + table_df['ID'].astype(str)

#display(table_df)
writeTable(table_df,"PBITables")
logActivity('PBITables','Table processing completed')

StatementMeta(, 7f293312-2244-4a98-b187-e4258ae663d2, 13, Finished, Available, Finished)

log query=INSERT INTO amaser_dp700.testthislakehouseoption.docuschema.modellog (timeval,table_name,Message) VALUES (CURRENT_TIMESTAMP(),"PBITables","Table processing started")


amaser_dp700.testthislakehouseoption.docuschema.PBITables


log query=INSERT INTO amaser_dp700.testthislakehouseoption.docuschema.modellog (timeval,table_name,Message) VALUES (CURRENT_TIMESTAMP(),"PBITables","PBITables written to lakehouse")


log query=INSERT INTO amaser_dp700.testthislakehouseoption.docuschema.modellog (timeval,table_name,Message) VALUES (CURRENT_TIMESTAMP(),"PBITables","Table processing completed")


collect measure data from listed models and store in the lakehouse

In [ ]:
#get measure Data
logActivity('MeasureList','MeasureList processing started')
dax_query = "EVALUATE INFO.Measures()"
measure_df = getMetaData(dax_query,'MeasureName','measures')

#display(measure_df)

writeTable(measure_df,"MeasureList")
logActivity('MeasureList','MeasureList processing completed')

StatementMeta(, 7f293312-2244-4a98-b187-e4258ae663d2, 14, Finished, Available, Finished)

log query=INSERT INTO amaser_dp700.testthislakehouseoption.docuschema.modellog (timeval,table_name,Message) VALUES (CURRENT_TIMESTAMP(),"MeasureList","MeasureList processing started")


amaser_dp700.testthislakehouseoption.docuschema.MeasureList


log query=INSERT INTO amaser_dp700.testthislakehouseoption.docuschema.modellog (timeval,table_name,Message) VALUES (CURRENT_TIMESTAMP(),"MeasureList","MeasureList written to lakehouse")


log query=INSERT INTO amaser_dp700.testthislakehouseoption.docuschema.modellog (timeval,table_name,Message) VALUES (CURRENT_TIMESTAMP(),"MeasureList","MeasureList processing completed")



get role data (if any) from all selected models, and store it in the lakehouse

In [ ]:
#get role Data
logActivity('PBIRoles','Role processing started')
dax_query = "EVALUATE INFO.Roles()"
role_df = getMetaData(dax_query,'RoleName','roles')

#display(role_df)

writeTable(role_df,"PBIRoles")
logActivity('PBIRoles','Role processing completed')

StatementMeta(, 7f293312-2244-4a98-b187-e4258ae663d2, 15, Finished, Available, Finished)

log query=INSERT INTO amaser_dp700.testthislakehouseoption.docuschema.modellog (timeval,table_name,Message) VALUES (CURRENT_TIMESTAMP(),"PBIRoles","Role processing started")


amaser_dp700.testthislakehouseoption.docuschema.PBIRoles


log query=INSERT INTO amaser_dp700.testthislakehouseoption.docuschema.modellog (timeval,table_name,Message) VALUES (CURRENT_TIMESTAMP(),"PBIRoles","PBIRoles written to lakehouse")


log query=INSERT INTO amaser_dp700.testthislakehouseoption.docuschema.modellog (timeval,table_name,Message) VALUES (CURRENT_TIMESTAMP(),"PBIRoles","Role processing completed")


get data about any partitions.  This may only be important in regards to incremental refresh, but either way, the data is available in the documentation lakehouse

In [ ]:
#get partition Data
logActivity('Partitions','Partition processing started')
dax_query = """
EVALUATE
VAR __partitions =
    INFO.PARTITIONS ()
VAR __refreshpolicies =
    SELECTCOLUMNS (
        INFO.REFRESHPOLICIES (),
        \"TableID\", [TableID],
        \"SourceExpression\", [SourceExpression]
    )
VAR __combined =
    NATURALLEFTOUTERJOIN ( __partitions, __refreshpolicies )
RETURN
    __combined
"""
partition_df = getMetaData(dax_query,'PartitionName','partitions')

#add extra columns for joins, and cleanup unused columns
partition_df['compoundkey'] = partition_df['SemanticModel'] + partition_df['TableID'].astype(str)
partition_df =pd.merge(partition_df,table_df[['compoundkey','TableName']],left_on='compoundkey',right_on='compoundkey',how='left')
partition_df['Source'] = np.where(partition_df['QueryDefinition'].isnull(),partition_df['SourceExpression'],partition_df['QueryDefinition'])
partition_df = partition_df.drop(['QueryDefinition','SourceExpression'],axis=1)
#display(partition_df)

writeTable(partition_df,"Partitions")
logActivity('Partitions','Partition processing completed')

StatementMeta(, 7f293312-2244-4a98-b187-e4258ae663d2, 16, Finished, Available, Finished)

log query=INSERT INTO amaser_dp700.testthislakehouseoption.docuschema.modellog (timeval,table_name,Message) VALUES (CURRENT_TIMESTAMP(),"Partitions","Partition processing started")


amaser_dp700.testthislakehouseoption.docuschema.Partitions


log query=INSERT INTO amaser_dp700.testthislakehouseoption.docuschema.modellog (timeval,table_name,Message) VALUES (CURRENT_TIMESTAMP(),"Partitions","Partitions written to lakehouse")


log query=INSERT INTO amaser_dp700.testthislakehouseoption.docuschema.modellog (timeval,table_name,Message) VALUES (CURRENT_TIMESTAMP(),"Partitions","Partition processing completed")


get expression data.  This includes data sources (as shown in TE) and any parameters that may be defined.

In [ ]:
#get expression Data
logActivity('Expression','Expression processing started')
dax_query = "EVALUATE INFO.Expressions()"
expression_df = getMetaData(dax_query,'ExpressionName','expression')

#display(expression_df)

writeTable(expression_df,"Expression")
logActivity('Expression','Expression processing completed')

StatementMeta(, 7f293312-2244-4a98-b187-e4258ae663d2, 17, Finished, Available, Finished)

log query=INSERT INTO amaser_dp700.testthislakehouseoption.docuschema.modellog (timeval,table_name,Message) VALUES (CURRENT_TIMESTAMP(),"Expression","Expression processing started")


amaser_dp700.testthislakehouseoption.docuschema.Expression


log query=INSERT INTO amaser_dp700.testthislakehouseoption.docuschema.modellog (timeval,table_name,Message) VALUES (CURRENT_TIMESTAMP(),"Expression","Expression written to lakehouse")


log query=INSERT INTO amaser_dp700.testthislakehouseoption.docuschema.modellog (timeval,table_name,Message) VALUES (CURRENT_TIMESTAMP(),"Expression","Expression processing completed")


get hierarchy data (if any)

In [ ]:
#get hierarchy Data
logActivity('Hierarchies','Hierarchy processing started')
dax_query = "EVALUATE INFO.Hierarchies()"
hierarchy_df = getMetaData(dax_query,'HierarchyName','hierarchy')

#display(final_df)
#add colulmn for join with table
hierarchy_df['compoundkey'] = hierarchy_df['SemanticModel'] + hierarchy_df['TableID'].astype(str)

hierarchy_df =pd.merge(hierarchy_df,table_df[['compoundkey','TableName']],left_on='compoundkey',right_on='compoundkey',how='left')
#display(hierarchy_df)

writeTable(hierarchy_df,"Hierarchies")
logActivity('Hierarchies','Hierarchy processing completed')

StatementMeta(, 7f293312-2244-4a98-b187-e4258ae663d2, 18, Finished, Available, Finished)

log query=INSERT INTO amaser_dp700.testthislakehouseoption.docuschema.modellog (timeval,table_name,Message) VALUES (CURRENT_TIMESTAMP(),"Hierarchies","Hierarchy processing started")


amaser_dp700.testthislakehouseoption.docuschema.Hierarchies


log query=INSERT INTO amaser_dp700.testthislakehouseoption.docuschema.modellog (timeval,table_name,Message) VALUES (CURRENT_TIMESTAMP(),"Hierarchies","Hierarchies written to lakehouse")


log query=INSERT INTO amaser_dp700.testthislakehouseoption.docuschema.modellog (timeval,table_name,Message) VALUES (CURRENT_TIMESTAMP(),"Hierarchies","Hierarchy processing completed")


get column data from selected models

In [ ]:
#get column Data
logActivity('PBIColumns','Column processing started')
dax_query = "EVALUATE INFO.Columns()"
columns_df = getMetaData(dax_query,'ColumnName','column')

#add compound key for table join.  add colulmn to determine calculated columns (used for slicer)
columns_df['compoundkey'] = columns_df['SemanticModel'] + columns_df['TableID'].astype(str)
columns_df =pd.merge(columns_df,table_df[['compoundkey','TableName']],left_on='compoundkey',right_on='compoundkey',how='left')
columns_df['ColumnName'] = columns_df['ExplicitName'].combine_first(columns_df['InferredName'])
del columns_df['ExplicitName']
del columns_df['InferredName']
columns_df['ColumnType'] = np.where(columns_df['Expression'].isnull(),'Standard','Calculated')
#display(columns_df)

writeTable(columns_df,"PBIColumns")
logActivity('PBIColumns','Column processing completed')

StatementMeta(, 7f293312-2244-4a98-b187-e4258ae663d2, 19, Finished, Available, Finished)

log query=INSERT INTO amaser_dp700.testthislakehouseoption.docuschema.modellog (timeval,table_name,Message) VALUES (CURRENT_TIMESTAMP(),"PBIColumns","Column processing started")


amaser_dp700.testthislakehouseoption.docuschema.PBIColumns


log query=INSERT INTO amaser_dp700.testthislakehouseoption.docuschema.modellog (timeval,table_name,Message) VALUES (CURRENT_TIMESTAMP(),"PBIColumns","PBIColumns written to lakehouse")


log query=INSERT INTO amaser_dp700.testthislakehouseoption.docuschema.modellog (timeval,table_name,Message) VALUES (CURRENT_TIMESTAMP(),"PBIColumns","Column processing completed")


get info on any calculation groups from selected models

In [ ]:
#get calculation groups
logActivity('CalculationItems','Calculation Item processing started')
dax_query = "EVALUATE INFO.CalculationGroups()"
calcgroup_df = getMetaData(dax_query,'CalculationGroupName','calcgroup')
calcgroup_df['compoundkey'] = calcgroup_df['SemanticModel'] + calcgroup_df['TableID'].astype(str)
calcgroup_df['groupcompoundkey'] = calcgroup_df['SemanticModel'] + calcgroup_df['ID'].astype(str)
calcgroup_df =pd.merge(calcgroup_df,table_df[['compoundkey','TableName']],left_on='compoundkey',right_on='compoundkey',how='left')
calcgroup_df['CalculationGroupName'] = calcgroup_df['TableName']
#display(calcgroup_df)

#get calcitems
dax_query = "EVALUATE INFO.CalculationItems()"
items_df = getMetaData(dax_query,'CalculationItemName','calcitem')
items_df['compoundkey'] = items_df['SemanticModel'] + items_df['CalculationGroupID'].astype(str)
items_df = pd.merge(items_df,calcgroup_df[['groupcompoundkey','CalculationGroupName']],left_on='compoundkey',right_on='groupcompoundkey',how='left')
#display(items_df)

writeTable(items_df,"CalculationItems")
logActivity('CalculationItems','Calculation Item processing completed')

StatementMeta(, 7f293312-2244-4a98-b187-e4258ae663d2, 20, Finished, Available, Finished)

log query=INSERT INTO amaser_dp700.testthislakehouseoption.docuschema.modellog (timeval,table_name,Message) VALUES (CURRENT_TIMESTAMP(),"CalculationItems","Calculation Item processing started")


amaser_dp700.testthislakehouseoption.docuschema.CalculationItems


log query=INSERT INTO amaser_dp700.testthislakehouseoption.docuschema.modellog (timeval,table_name,Message) VALUES (CURRENT_TIMESTAMP(),"CalculationItems","CalculationItems written to lakehouse")


log query=INSERT INTO amaser_dp700.testthislakehouseoption.docuschema.modellog (timeval,table_name,Message) VALUES (CURRENT_TIMESTAMP(),"CalculationItems","Calculation Item processing completed")


get Relationship data from selected models.  this does not include relationships for this documentation model 

In [ ]:
#get relationships
logActivity('Relationships','Relationship processing started')
dax_query = "EVALUATE INFO.Relationships()"
relationship_df = getMetaData(dax_query,'RelationshipName','relationship')
# clean up colulmns for to/from relationships.  
relationship_df['FromCompoundKey'] = relationship_df['SemanticModel'] + relationship_df['FromTableID'].astype(str)
relationship_df['ToCompoundKey'] = relationship_df['SemanticModel'] + relationship_df['ToTableID'].astype(str)
relationship_df =pd.merge(relationship_df,table_df[['compoundkey','TableName','TableType']],left_on='FromCompoundKey',right_on='compoundkey',how='left')
relationship_df = relationship_df.rename(columns={'TableName':'FromTableName','TableType':'FromTableType'})
relationship_df =pd.merge(relationship_df,table_df[['compoundkey','TableName','TableType']],left_on='ToCompoundKey',right_on='compoundkey',how='left')
relationship_df = relationship_df.rename(columns={'TableName':'ToTableName','TableType':'ToTableType'})
relationship_df['FromCardinality'] = relationship_df['FromCardinality'].astype(str)
relationship_df['FromCardinality'] = relationship_df['FromCardinality'].replace('2','Many')
relationship_df['ToCardinality'] = relationship_df['ToCardinality'].astype(str)
relationship_df['ToCardinality'] = relationship_df['ToCardinality'].replace('2','Many')
####################################################################################################################
#define parameters. you do not need to edit this section
FromCondition = [
    (relationship_df['FromTableType'] == 'Fact'),
    (relationship_df['FromTableType'] == 'Dimension')
]
ToCondition = [
    (relationship_df['ToTableType'] == 'Fact'),
    (relationship_df['ToTableType'] == 'Dimension')
]
values = [factcolor,dimcolor]
###########################################################################################
relationship_df['ToColor'] = np.select(ToCondition,values,default=defaultcolor)
relationship_df['FromColor'] = np.select(FromCondition,values,default=defaultcolor)
#display(relationship_df)

writeTable(relationship_df,"Relationships")
logActivity('Relationships','Relationship processing completed')

StatementMeta(, 7f293312-2244-4a98-b187-e4258ae663d2, 21, Finished, Available, Finished)

log query=INSERT INTO amaser_dp700.testthislakehouseoption.docuschema.modellog (timeval,table_name,Message) VALUES (CURRENT_TIMESTAMP(),"Relationships","Relationship processing started")


amaser_dp700.testthislakehouseoption.docuschema.Relationships


log query=INSERT INTO amaser_dp700.testthislakehouseoption.docuschema.modellog (timeval,table_name,Message) VALUES (CURRENT_TIMESTAMP(),"Relationships","Relationships written to lakehouse")


log query=INSERT INTO amaser_dp700.testthislakehouseoption.docuschema.modellog (timeval,table_name,Message) VALUES (CURRENT_TIMESTAMP(),"Relationships","Relationship processing completed")
